In [11]:

treshold = 0.52

# file = '../evaluation/multi_commits_rly_false_pairs_result.txt'
# file = 'evaluation/multi_commits_msr_result.txt'
file = 'detection/msr_multi_commits_no_repeat_newret.txt'
# file = 'detection/multi_commits_second_nondup_part_newret-c.txt'

with open(file) as f:
    ps = f.readlines()

second_r = ['elastic/elasticsearch', 'cocos2d/cocos2d-x', 'emberjs/ember.js', 'django/django', 'rails/rails', 'angular/angular.js', 'dotnet/corefx', 'ceph/ceph', 'ansible/ansible', 'hashicorp/terraform', 'joomla/joomla-cms', 'facebook/react', 'JuliaLang/julia', 'docker/docker']

pro_save = 0

save = []
for p in ps:
    p1, hist = p.strip()[:-1].split('[')
    r, n1, n2, ori, det, delta, zz = p1.split()
    
    if r not in second_r:
        continue
    
    if det == '2':
        continue

    hist = [float(h) for h in hist.split(',')]
    
    
    if len(hist) == 1:
        continue
    
    pro_save += 1
    
    s = 0
    for i in range(len(hist)):
        if hist[i] >= treshold:
            s = len(hist) - i - 1
            break
    save.append(s)

print('could_save', len(save))

print('all=', pro_save)

no_zero = 0
for i in save:
    if i > 0:
        no_zero += 1

print('no_zero=', no_zero / pro_save)

print(sum(save))
print(sum(save) / pro_save)


could_save 105
all= 105
no_zero= 0.5904761904761905
156
1.4857142857142858


In [ ]:
import sys
sys.path.append('/home/luyao/PR_get/INTRUDE')

from git import *

with open('data/clf/second_nondup.txt') as f:
    pairs = f.readlines()

w = 0

out = open('data/multi_commits_second_nondup.txt', 'w')

pairs = sorted(pairs, key=lambda x: x.split()[0])
last_repo = None

for pair in pairs:
    repo, num1, num2 = pair.split()
    
    #print(repo, num1, num2)
    '''
    if repo != last_repo:
        last_repo = repo
        init_model_with_repo(repo)
    '''
    
    p1 = get_pull(repo, num1)
    p2 = get_pull(repo, num2)
    
    cl1 = get_pull_commit(p1)
    cl2 = get_pull_commit(p2)
    
    
    #t = get_pr_sim_vector(p1, p2)
    #o = m.predict_proba([t])[0][1]
    
    if (len(cl1) == 1) and (len(cl2) == 1):
        continue
        
    if (len(cl1) == 0) or (len(cl2) == 0):
        continue
    
    if (len(cl1) > 100) or (len(cl2) > 100):
        continue

    if check_large(p1) or check_large(p2):
        continue
    
    print(pair.strip(), file=out)
    out.flush()

    w += 1
    if w % 100 == 0:
        print(w)

out.close()
print(w)

In [1]:
r2 = set()
with open('data/clf/second_msr_pairs_nolarge.txt') as f:
	for t in f.readlines():
		r, n1, n2 = t.split()
		r2.add(r)

print(r2)


{'angular/angular.js', 'facebook/react', 'rails/rails', 'hashicorp/terraform', 'joomla/joomla-cms', 'docker/docker', 'ceph/ceph', 'dotnet/corefx', 'django/django', 'cocos2d/cocos2d-x', 'ansible/ansible', 'JuliaLang/julia', 'elastic/elasticsearch', 'emberjs/ember.js'}


In [17]:
import sys
sys.path.append('/home/luyao/PR_get/INTRUDE')

import os
from git import *
from sklearn.utils import shuffle

repo = 'tensorflow/tensorflow'
file = 'evaluation/' + repo.replace('/', '_') + '_select.txt'

if os.path.exists(file):
    raise Exception('file already exists!')

pulls = get_repo_info(repo, 'pull')

ps = shuffle(pulls)

cnt = 0
with open(file, 'w') as f:
    for p in ps:
        if not check_large(p):
            cnt += 1
            print(p['number'], file=f)
            
            if cnt == 200:
                break

        


In [21]:
from git import *

repo = 'rails/rails'
file = 'evaluation/rails_rails_select.txt'
with open(file) as f:
    for n in f.readlines():
        p = get_pull(repo, n.strip())
        if check_large(p):
            print(n)
